## Aanpak
Train a model to test if a given (summarized) text confirms or debunks a statement (or is irrelevant)

- Labelling:
    - Mix hypothesis and premises
    - News articles, title + article, positive label
- Fine tune on NLI model

- https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb
- https://huggingface.co/blog/lora
- https://github.com/adapter-hub/adapter-transformers

- https://huggingface.co/datasets/snli

In [ ]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer
from datasets import Dataset

In [ ]:
base_model = 'cross-encoder/nli-distilroberta-base'

In [ ]:
test_data = pd.read_csv('../../data/temp/JoeBidentookhomeclassifieddocumentsafterleavingthevice-presidency.csv', sep='\t')

test_data['label'] = np.random.randint(0, 3, test_data.shape[0])
test_data['hypothesis'] = "Joe Biden took home classified documents after leaving the vice-presidency"
test_data['premise'] = test_data['summary']

test_data = test_data[['label', 'hypothesis', 'premise']]
test_data = pd.concat([test_data]*100)

test_data = Dataset.from_pandas(test_data)
test_data[5]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['hypothesis'], examples['premise'], padding="max_length", truncation=True)

test_data = test_data.map(tokenize_function, batched=True)

In [ ]:
test_data = test_data.train_test_split(test_size=0.2)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=3)

In [ ]:
metric_name = 'f1'
task = 'mnli'
batch_size = 16
model_name = base_model.split("/")[-1]

args = TrainingArguments(
    f"../data/checkpoints/{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## Test: use adapters

In [ ]:
# Add a new adapter
model.add_adapter("test_adapter")
# Add a matching classification head
model.add_classification_head(
    "test_adapter",
    num_labels=3,
    id2label={ 0:"contradiction", 1: "neutral", 2:"entailment"}
  )
# Activate the adapter
model.train_adapter("test_adapter")

## Train

In [ ]:
validation_key = "test"

trainer = Trainer(
    model,
    args,
    train_dataset=test_data["train"],
    eval_dataset=test_data[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()